# FastQC
# What does this do?

### Run FastQC
- Get list of sample IDs for all alignments.bam files on Google Cloud Storage (GCS)
- Get list of sample IDs for all alignments.bam.fastqc_data.txt files on GCS
- Compare lists to find samples missing fastqc data files
- Use make-batch-tsv-from-input-sample.py to generate dsub task file
- Run dsub to generate missing alignments.bam.fastqc_data.txt files

### Convert FastQC result files to CSV format
- Get list of all fastqc_data.txt files
- Get list of all fastqc_data.txt.csv files
- Convert each to lists of sample IDs
- Compare lists to find samples that are missing fastqc_data.txt.csv files
- Use make-batch-tsv-from-input-file.py to generate dsub task file
- Run dsub to generate missing fastqc_data.txt.csv files

### Concatenate CSV Files
- Get list of all fastqc_data.txt.csv files
- Run dsub to concatenate all fastqc_data.txt.csv files

# Code

## 0. Check that environment variables have been loaded correctly

Environment variables are imported from the mvp-profile.sh file. If this echo command does not return anything, try sourcing it from the command console. If any of the values are incorrect, change them in mvp-profile.sh, save it, and source it again.

In [ ]:
# source mvp_profile.sh
echo "Date stamp: ${date_stamp}"
echo "Home directory: ${mvp_hub}"
echo "Project: ${mvp_project}"
echo "Bucket: ${mvp_bucket}"
echo "Zone: ${mvp_zone}"

## 1. Run samtools flagstat

#### Create file accounting directory it doesn't already exist

In [ ]:
accounting_dir="${mvp_hub}/fastqc-bam/file-accounting/${date_stamp}"
mkdir -p ${accounting_dir}

dsub_inputs_dir="${mvp_hub}/fastqc-bam/dsub-inputs"

#### Get list of sample IDs for gvcf files that already exist on Google Cloud Storage

In [ ]:
gsutil ls gs://${mvp_bucket}/data/bina-deliverables/*/*/Recalibration/alignments.bam \
    > ${accounting_dir}/gs-bam-${date_stamp}.txt
cut -d'/' -f6 ${accounting_dir}/gs-bam-${date_stamp}.txt \
    > ${accounting_dir}/gs-bam-sample-ids-${date_stamp}.txt

#### Get list sample IDs for FastQC files that already exist on Google Cloud Storage

In [ ]:
gsutil ls gs://${mvp_bucket}/dsub/fastqc-bam/fastqc/objects/*_alignments.bam.fastqc_data.txt \
    > ${accounting_dir}/gs-fastqc-data-${date_stamp}.txt
cut -d '/' -f8 ${accounting_dir}/gs-fastqc-data-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-fastqc-data-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing FastQC files

In [ ]:
diff --new-line-format="" --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-bam-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-fastqc-data-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-fastqc-data-missing-sample-ids-${date_stamp}.txt
grep -F \
    -f ${accounting_dir}/gs-fastqc-data-missing-sample-ids-${date_stamp}.txt \
    ${accounting_dir}/gs-bam-${date_stamp}.txt \
    > ${accounting_dir}/gs-fastqc-data-missing-${date_stamp}.txt

#### Create dsub TSV task file to generate missing vcfstats files

In [ ]:
${mvp_hub}/bin/make-batch-tsv-from-input-sample.py \
    -i ${accounting_dir}/gs-fastqc-data-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/fastqc/gs-fastqc-data-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/fastqc-bam/fastqc/objects \
    -s alignments.bam.fastqc_data.txt

#### Run FastQC dsub tasks

In [ ]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/fastqc-bam/fastqc/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/fastqc:1.01 \
    --disk-size 500 \
    --script ${mvp_hub}/fastqc-bam/dsub-scripts/fastqc.sh \
    --tasks ${mvp_hub}/fastqc-bam/dsub-inputs/fastqc/gs-fastqc-data-missing-${date_stamp}.tsv ${dsub_range}\
    --dry-run

## 2. Convert FastQC data files to CSV format

#### Get list of sample IDs for FastQC data files that already exist on Google Cloud Storage

In [ ]:
gsutil ls gs://${mvp_bucket}/dsub/fastqc-bam/fastqc/objects/*alignments.bam.fastqc_data.txt \
  > ${accounting_dir}/gs-fastqc-bam-txt-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-fastqc-bam-txt-${date_stamp}.txt \
  | cut -d'_' -f1 > ${accounting_dir}/gs-fastqc-bam-txt-sample-ids-${date_stamp}.txt

#### Get list of sample IDs for FastQC CSV files that already exist on Google Cloud Storage

In [ ]:
gsutil ls gs://${mvp_bucket}/dsub/fastqc-bam/text-to-table/objects/*alignments.bam.fastqc_data.txt.csv \
    > ${accounting_dir}/gs-fastqc-bam-csv-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-fastqc-bam-csv-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-fastqc-bam-csv-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [ ]:
diff \
  --new-line-format="" \
  --unchanged-line-format "" \
  <(sort ${accounting_dir}/gs-fastqc-bam-txt-sample-ids-${date_stamp}.txt) \
  <(sort ${accounting_dir}/gs-fastqc-bam-csv-sample-ids-${date_stamp}.txt) \
  > ${accounting_dir}/gs-fastqc-bam-csv-sample-ids-missing-${date_stamp}.txt
grep -F \
  -f ${accounting_dir}/gs-fastqc-bam-csv-sample-ids-missing-${date_stamp}.txt \
  ${accounting_dir}/gs-fastqc-bam-txt-${date_stamp}.txt \
  > ${accounting_dir}/gs-fastqc-bam-csv-missing-${date_stamp}.txt

#### Convert file list to dsub TSV files

In [ ]:
${mvp_hub}/bin/make-batch-tsv-from-input-file.py \
    -i ${accounting_dir}/gs-bina-fastqc-bam-csv-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/text-to-table/gs-fastqc-bam-csv-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/fastqc-bam/text-to-table/objects \
    -s csv \
    -c fastqc \
    -e fastqc-bam-${date_stamp}

#### Run dsub task

In [ ]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/fastqc-bam/text-to-table/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --command 'text2table -s ${SCHEMA} -o ${OUTPUT} -v series=${SERIES},sample=${SAMPLE_ID} ${INPUT}' \
    --tasks ${dsub_inputs_dir}/text-to-table/gs-fastqc-bam-csv-missing-${date_stamp}.tsv ${mvp_range} \
    #--dry-run

## 3. Concatenate CSV Files

#### Get new list of completed results files

In [ ]:
gsutil ls gs://${mvp_bucket}/dsub/fastqc-bam/text-to-table/objects/*alignments.bam.fastqc_data.txt.csv \
    > ${accounting_dir}/gs-fastqc-bam-csv-${date_stamp}.txt

#### Run dsub task

In [ ]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/fastqc-bam/concat/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --disk-size 100 \
    --vars-include-wildcards \
    --input INPUT_FILES=gs://${mvp_bucket}/dsub/fastqc-bam/text-to-table/objects/*alignments.bam.fastqc_data.txt.csv \
    --output CONCAT_FILE=gs://${mvp_bucket}/dsub/fastqc-bam/concat/objects/concat_alignments.bam.fastqc_data.txt.csv \
    --command 'cat ${INPUT_FILES} > ${CONCAT_FILE}' \
    #--dry-run